In [1]:
import pandas as pd
import pickle
import numpy as np
import torch
from torch import nn
import os 
import fastai
import fastai
from fastai.text import *

In [2]:
PATH="/home/kirana/Documents/phd"
DATAPATH="/home/kirana/Documents/phd/data/experiment/YELP_BINARY"

## Dataset

In [3]:
[df_train,df_valid,itos, train_tokens, valid_tokens, trn_lm, val_lm]=pickle.load(open(f'{DATAPATH}/inter/dfs_tokens_fastai.pkl','rb'))

In [4]:
type(df_train)

pandas.core.frame.DataFrame

In [5]:
df_train.head()

,text,label,dstype,words,tokens
241568,"Went tonight, and had an AWFUL time due to a f...",0,train,"[ \n , xxbos, xxfld, 1, xxmaj, went, tonight, ...","[28, 29, 30, 25, 2, 118, 1051, 5, 6, 35, 81, 2..."
302519,Go. Hurry and go. Mexican food with such an am...,1,train,"[ \n , xxbos, xxfld, 1, xxmaj, go, ., xxmaj, h...","[28, 29, 30, 25, 2, 78, 3, 2, 2217, 6, 78, 3, ..."
7005,"I used to love coming to foxy nails, and Missy...",0,train,"[ \n , xxbos, xxfld, 1, i, used, to, love, com...","[28, 29, 30, 25, 7, 323, 8, 141, 368, 8, 27370..."
20836,"This place has good italian food for phoenix, ...",0,train,"[ \n , xxbos, xxfld, 1, xxmaj, this, place, ha...","[28, 29, 30, 25, 2, 22, 46, 113, 48, 682, 44, ..."
808,Worst climbing gym I have ever been to. Custom...,0,train,"[ \n , xxbos, xxfld, 1, xxmaj, worst, climbing...","[28, 29, 30, 25, 2, 403, 5685, 1174, 7, 34, 17..."


In [6]:
df=pd.concat([df_train,df_valid],axis=0)

In [7]:
df['dstype'].value_counts()

train    560000
test      38000
Name: dstype, dtype: int64

In [8]:
df_train=df.loc[df['dstype']!='test']
df_test=df.loc[df['dstype']=='test']

In [9]:
df_train.shape, df_test.shape

((560000, 5), (38000, 5))

In [10]:
df_train.head()

,text,label,dstype,words,tokens
241568,"Went tonight, and had an AWFUL time due to a f...",0,train,"[ \n , xxbos, xxfld, 1, xxmaj, went, tonight, ...","[28, 29, 30, 25, 2, 118, 1051, 5, 6, 35, 81, 2..."
302519,Go. Hurry and go. Mexican food with such an am...,1,train,"[ \n , xxbos, xxfld, 1, xxmaj, go, ., xxmaj, h...","[28, 29, 30, 25, 2, 78, 3, 2, 2217, 6, 78, 3, ..."
7005,"I used to love coming to foxy nails, and Missy...",0,train,"[ \n , xxbos, xxfld, 1, i, used, to, love, com...","[28, 29, 30, 25, 7, 323, 8, 141, 368, 8, 27370..."
20836,"This place has good italian food for phoenix, ...",0,train,"[ \n , xxbos, xxfld, 1, xxmaj, this, place, ha...","[28, 29, 30, 25, 2, 22, 46, 113, 48, 682, 44, ..."
808,Worst climbing gym I have ever been to. Custom...,0,train,"[ \n , xxbos, xxfld, 1, xxmaj, worst, climbing...","[28, 29, 30, 25, 2, 403, 5685, 1174, 7, 34, 17..."


In [11]:
type(df_train['tokens'])

pandas.core.series.Series

In [12]:
bs=52 # 52 - Jeremey, 20 - default

In [13]:
bptt= 70 #70 - Jeremey, 35 - default

In [14]:
df_train.head()

,text,label,dstype,words,tokens
241568,"Went tonight, and had an AWFUL time due to a f...",0,train,"[ \n , xxbos, xxfld, 1, xxmaj, went, tonight, ...","[28, 29, 30, 25, 2, 118, 1051, 5, 6, 35, 81, 2..."
302519,Go. Hurry and go. Mexican food with such an am...,1,train,"[ \n , xxbos, xxfld, 1, xxmaj, go, ., xxmaj, h...","[28, 29, 30, 25, 2, 78, 3, 2, 2217, 6, 78, 3, ..."
7005,"I used to love coming to foxy nails, and Missy...",0,train,"[ \n , xxbos, xxfld, 1, i, used, to, love, com...","[28, 29, 30, 25, 7, 323, 8, 141, 368, 8, 27370..."
20836,"This place has good italian food for phoenix, ...",0,train,"[ \n , xxbos, xxfld, 1, xxmaj, this, place, ha...","[28, 29, 30, 25, 2, 22, 46, 113, 48, 682, 44, ..."
808,Worst climbing gym I have ever been to. Custom...,0,train,"[ \n , xxbos, xxfld, 1, xxmaj, worst, climbing...","[28, 29, 30, 25, 2, 403, 5685, 1174, 7, 34, 17..."


In [15]:
df_train['label'].value_counts()

1    280000
0    280000
Name: label, dtype: int64

In [16]:
df_valid['label'].value_counts()

0    29986
1    29814
Name: label, dtype: int64

In [17]:
itos[:5]

['_unk_', '_pad_', 'xxmaj', '.', 'the']

In [18]:
padlen=1400
padding_idx=1

In [19]:
df_train['n_tok']=df_train['tokens'].apply(len)

/home/kirana/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [20]:
df_train['n_tok'].describe()

count    560000.000000
mean        174.079913
std         156.012100
min           5.000000
25%          70.000000
50%         128.000000
75%         225.000000
max        1645.000000
Name: n_tok, dtype: float64

In [21]:
df_train.sort_values(by='n_tok', inplace=True)

/home/kirana/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [22]:
df_valid['n_tok']=df_valid['tokens'].apply(len)
df_valid.sort_values(by='n_tok', inplace=True)

In [23]:
df_test['n_tok']=df_test['tokens'].apply(len)
df_test.sort_values(by='n_tok', inplace=True)

/home/kirana/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/kirana/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [24]:
for i in range(0,20):
    x=df_train['tokens'].values[i]
    y=df_train['label'].values[i]    
    out=np.ones(padlen)
    if len(x) < padlen:
        out[:len(x)]=x
    else:
        out=x[:padlen]
    print (out)
    print (y)
    print (len(out))
    if i>10:
        break

[28. 29. 30. 25. ...  1.  1.  1.  1.]
0
1400
[28. 29. 30. 25. ...  1.  1.  1.  1.]
0
1400
[28. 29. 30. 25. ...  1.  1.  1.  1.]
0
1400
[28. 29. 30. 25. ...  1.  1.  1.  1.]
0
1400
[28. 29. 30. 25. ...  1.  1.  1.  1.]
0
1400
[28. 29. 30. 25. ...  1.  1.  1.  1.]
0
1400
[28. 29. 30. 25. ...  1.  1.  1.  1.]
0
1400
[28. 29. 30. 25. ...  1.  1.  1.  1.]
0
1400
[28. 29. 30. 25. ...  1.  1.  1.  1.]
0
1400
[28. 29. 30. 25. ...  1.  1.  1.  1.]
0
1400
[28. 29. 30. 25. ...  1.  1.  1.  1.]
0
1400
[28. 29. 30. 25. ...  1.  1.  1.  1.]
0
1400


In [25]:
df_train.shape

(560000, 6)

In [26]:
df_train['label'].value_counts(), df_valid['label'].value_counts(),df_test['label'].value_counts()

(1    280000
 0    280000
 Name: label, dtype: int64, 0    29986
 1    29814
 Name: label, dtype: int64, 1    19000
 0    19000
 Name: label, dtype: int64)

In [27]:
df_train.shape, df_valid.shape

((560000, 6), (59800, 6))

In [28]:
padlen

1400

In [29]:
class ds_sentiment (Dataset):
    def __init__ (self,df,bs,padlen=64,xvar='tokens',yvar='label',len_var='n_tok',padding_idx=1):
        self.x,self.y,self.padlen,self.padding_idx,self.len_var,self.bs=\
            df[xvar],df[yvar],padlen,padding_idx,df[len_var],bs
        self.len_var=self.len_var.clip(0,padlen)
    
    def pad (self,x):
        out=np.ones(self.padlen)*self.padding_idx
        out=out.astype(int)
        if len(x)>=self.padlen:
            out[:]=x[:self.padlen]
        else:
            out[:len(x)]=x
        return out
        
    def __getitem__(self,idx):
        #import pdb
        #pdb.set_trace()
        return self.pad(self.x.iloc[idx]),self.y.iloc[idx],self.len_var.iloc[idx]
    
    def __len__(self):
        return len(self.x)        


In [30]:
#padlen=max(df_train['n_tok'])
padlen=1400

In [31]:
padlen

1400

In [32]:
dstrain=ds_sentiment(df_train,bs,padlen)
dsvalid=ds_sentiment(df_valid,bs,padlen)
dstest=ds_sentiment(df_test,bs,padlen)

In [33]:
dltrain=DataLoader(dstrain,bs,True)
dlvalid=DataLoader(dsvalid,bs,False)
dltest=DataLoader(dstest,bs,False)

In [34]:
for xb,yb,xlen in dltrain:
    break

In [35]:
xb, yb, xlen

(tensor([[28, 29, 30,  ...,  1,  1,  1],
         [28, 29, 30,  ...,  1,  1,  1],
         [28, 29, 30,  ...,  1,  1,  1],
         ...,
         [28, 29, 30,  ...,  1,  1,  1],
         [28, 29, 30,  ...,  1,  1,  1],
         [28, 29, 30,  ...,  1,  1,  1]]),
 tensor([1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1,
         0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0,
         0, 0, 0, 1]),
 tensor([285, 131, 129, 454, 155, 160, 365,  80, 288, 358, 418,  45,  17, 654,
         104,  32,  27, 175,  20,  58, 147, 288,  69, 156,  46,  75, 244, 104,
         154,  96, 315, 143, 168,  71, 350, 206,  41, 138, 211, 111, 130,  57,
          42,  30,  41,  21, 206,  38, 119,  49, 228, 229]))

## Model Architecture

In [36]:
from torch import nn
import torch
from torch.autograd import Variable

In [37]:
bs,bptt

(52, 70)

In [38]:
n_inp=len(itos)
n_emb=400 #650
n_hidden=400#400
n_layers= 2 # 2
dropout=0.5 # 0.5
wd=1e-7
bidirectional=True
dropout_e=0.2 # 0.5 - changing to 0.4, 0.3 or any dropout value did not make much difference
dropout_o=0.5 #0.5
n_out=1

In [39]:
xb.shape,xlen.shape

(torch.Size([52, 1400]), torch.Size([52]))

In [40]:
df_train['label'].value_counts()

1    280000
0    280000
Name: label, dtype: int64

In [41]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [42]:
itos[:5]

['_unk_', '_pad_', 'xxmaj', '.', 'the']

In [43]:
import torch

In [44]:
m = nn.AdaptiveAvgPool1d(5)
input = torch.randn(1, 64, 8)
input.size()

torch.Size([1, 64, 8])

In [45]:
m(input).size()

torch.Size([1, 64, 5])

In [46]:
class sentiment_classifier (nn.Module):
    def __init__(self,n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e=0.05,dropout=0.5,\
                 dropout_o=0.5,pretrain_mtx=None,n_out=1,padding_idx=1,n_filters=100,filter_sizes=[3,4,5]):
        super().__init__()
        self.n_inp,self.n_emb,self.n_hidden,self.n_layers,self.bidirectional,self.bs,self.device,self.pretrain_mtx,self.padding_idx=\
                            n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,pretrain_mtx,padding_idx
        self.n_out,self.n_filters,self.filter_sizes=n_out,n_filters,filter_sizes
        self.dropout_e,self.dropout,self.dropout_o=dropout_e,dropout,dropout_o
        
        self.create_architecture()
        if pretrain_mtx is not None:
            print (f'initializing glove with {pretrain_mtx.shape}')
            self.initialize_glove()
        self.init_hidden()
        self.criterion=nn.BCEWithLogitsLoss()
    
    def set_dropouts(self, dropout, dropout_o, dropout_e):
        self.dropout, self.dropout_o, self.dropout_e = dropout, dropout_o, dropout_e
    
    
    def freeze_embedding(self):
        self.encoder.weight.requires_grad=False
         
    def unfreeze_embedding(self):
        self.encoder.weight.requires_grad=True

    def initialize_glove(self):
        self.encoder.weight.data.copy_(torch.Tensor(self.pretrain_mtx))
    
    def init_hidden(self):
        # Initialize hidden
        self.hidden=(Variable(torch.zeros(self.n_layers,self.bs,self.n_hidden,requires_grad=False).to(self.device)),
                     Variable(torch.zeros(self.n_layers,self.bs,self.n_hidden,requires_grad=False).to(self.device)))
    

    def create_architecture(self):
        ###################################
        # Embedding layer - common to both
        ###################################
        self.dropout_enc=nn.Dropout(self.dropout_e)
        self.encoder=nn.Embedding(self.n_inp,self.n_emb,padding_idx=self.padding_idx)
        
        #######################################
        # For RNN #############################
        #######################################
        # Embedding Layer: Embedding layer just maps each word to an index. n_inp to n_emb mapping is all it does
            # input to this is of shape n_batch * n_seq
         # LSTM Layer
        self.lstm=nn.LSTM(self.n_emb,self.n_hidden,self.n_layers,batch_first=True,dropout=self.dropout,\
                          bidirectional=self.bidirectional)
          # embs are going to be of shape n_batch * n_seq * n_emb
        self.dropout_op=nn.Dropout(self.dropout_o)
        
        self.avg_pool1d=torch.nn.AdaptiveAvgPool1d(1)
        self.max_pool1d=torch.nn.AdaptiveMaxPool1d(1)
        
    
        #######################################
        # For CNN #############################
        #######################################    
        #embedding dimension is the "depth" of the filter and the number of tokens in the sentence is the width.
        self.conv_0=torch.nn.Conv1d (self.n_emb,self.n_filters,kernel_size=self.filter_sizes[0])
        self.conv_1=torch.nn.Conv1d (self.n_emb,self.n_filters,kernel_size=self.filter_sizes[1])
        self.conv_2=torch.nn.Conv1d(self.n_emb,self.n_filters,kernel_size=self.filter_sizes[2])
        
        self.fc=nn.Linear(len(self.filter_sizes)*self.n_filters+self.n_hidden*4,self.n_out)
        

        
    def forward (self,Xb,Yb,Xb_lengths):
        
        ####RNN PORTION
        embs=self.dropout_enc(self.encoder(Xb))
        if Xb.size(0) < self.bs:
            self.hidden=(self.hidden[0][:,:Xb.size(0),:].contiguous(),
            self.hidden[1][:,:Xb.size(0),:].contiguous())
        packed_embs = pack_padded_sequence(embs,Xb_lengths,batch_first=True, enforce_sorted=False)
        lstm_out,(hidden,cell)=self.lstm(packed_embs)
        lstm_out,lengths=pad_packed_sequence(lstm_out,batch_first=True)
        hidden = self.dropout_op(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        avg_pool=self.avg_pool1d(lstm_out.permute(0,2,1)).view(Xb.size(0),-1)
        max_pool=self.max_pool1d(lstm_out.permute(0,2,1)).view(Xb.size(0),-1)
        
        #CNN Portion
        new_embs=embs.permute(0,2,1)        
        conved_0=torch.relu(self.conv_0(new_embs))
        conved_1=torch.relu(self.conv_1(new_embs))
        conved_2=torch.relu(self.conv_2(new_embs)) 
        max_pool1d=torch.nn.MaxPool1d(conved_0.shape[2])
        pooled_0=max_pool1d(conved_0).squeeze(2)
        max_pool1d=torch.nn.MaxPool1d(conved_1.shape[2])
        pooled_1=max_pool1d(conved_1).squeeze(2)
        max_pool1d=torch.nn.MaxPool1d(conved_2.shape[2])
        pooled_2=max_pool1d(conved_2).squeeze(2)
        cat_cnn = self.dropout_op(torch.cat([pooled_0,pooled_1,pooled_2],dim=1))
        
        ## Concatenate
        big_out=torch.cat([cat_cnn,hidden,max_pool],dim=1)
        preds=self.fc(big_out)

        loss=self.criterion(preds,Yb.contiguous().float().view(-1,1))

        
        return preds.view(-1),loss
        
    

In [47]:
def accuracy_binomial(preds,actual, device="cpu", cutoff=0.5):
    preds=torch.sigmoid(preds)
    zeros=torch.zeros(len(preds)).to(device)
    ones = torch.ones(len(preds)).to(device)

    preds=torch.where(preds>cutoff,ones,zeros)
    correct=torch.round(preds).long()==actual
    return correct.float().sum()/len(correct)

In [48]:
def accuracy_binomial(preds, y, device="cpu", cutoff=0.5):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    y=y.float()
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [49]:
device="cpu"

## Load pretrained model weights

In [51]:
pretrained_lm_weights=pickle.load(open(f'{DATAPATH}/inter/varybpttpretrained_lm_weights','rb'))

### Check if model forward works

In [52]:
n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e,dropout_o,dropout,n_out

(60002, 400, 400, 2, True, 52, 'cpu', 0.2, 0.5, 0.5, 1)

In [53]:
model_sentiment=sentiment_classifier (n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e=dropout_e,dropout=dropout,\
                 dropout_o=dropout_o,pretrain_mtx=pretrained_lm_weights,n_out=1,padding_idx=1)


initializing glove with (60002, 400)


In [54]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {count_parameters(model_sentiment):,} trainable parameters')

The model has 30,895,801 trainable parameters


In [55]:
model_sentiment

sentiment_classifier(
  (dropout_enc): Dropout(p=0.2)
  (encoder): Embedding(60002, 400, padding_idx=1)
  (lstm): LSTM(400, 400, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (dropout_op): Dropout(p=0.5)
  (avg_pool1d): AdaptiveAvgPool1d(output_size=1)
  (max_pool1d): AdaptiveMaxPool1d(output_size=1)
  (conv_0): Conv1d(400, 100, kernel_size=(3,), stride=(1,))
  (conv_1): Conv1d(400, 100, kernel_size=(4,), stride=(1,))
  (conv_2): Conv1d(400, 100, kernel_size=(5,), stride=(1,))
  (fc): Linear(in_features=1900, out_features=1, bias=True)
  (criterion): BCEWithLogitsLoss()
)

In [56]:
model_sentiment.forward(xb,yb,xlen)

(tensor([ 0.0827, -0.0579,  0.9597, -0.1018,  0.1197,  0.0780, -0.2450, -0.2263,
          0.0157,  0.1950,  0.1299, -0.2848, -0.0062, -0.0548,  0.2742,  0.2059,
         -0.0463,  0.0562, -0.1905, -0.1809,  0.3561,  0.1049, -0.0773,  0.0228,
         -0.1071, -0.0340, -0.1633,  0.0326, -0.1183,  0.0797,  0.0591,  0.1533,
          0.1626,  0.1081, -0.0023,  0.3720,  0.0615, -0.1105,  0.0874,  0.0034,
         -0.2936, -0.4263,  0.0211, -0.1663,  0.0574, -0.0150,  0.1894,  0.1630,
          0.4449, -0.1912, -0.6450,  0.6491], grad_fn=<ViewBackward>),
 tensor(0.7076, grad_fn=<BinaryCrossEntropyWithLogitsBackward>))

In [57]:
preds,loss=model_sentiment.forward(xb,yb,xlen)

In [58]:
preds.view(-1)

tensor([-0.3451,  0.1161,  0.4484,  0.1373, -0.1280, -0.3728,  0.2010,  0.2596,
        -0.2033,  0.0127,  0.2176,  0.0865,  0.1524, -0.2347, -0.0990,  0.0845,
        -0.2835,  0.2939,  0.1618,  0.1024,  0.1894,  0.1100,  0.0531,  0.2357,
        -0.2977, -0.0345,  0.0411, -0.1780,  0.2834,  0.3831,  0.5624,  0.3275,
        -0.0502,  0.0553, -0.2646,  0.0862, -0.0816, -0.1093, -0.0842, -0.1010,
        -0.0075,  0.2538, -0.1554,  0.0840, -0.1627, -0.4590,  0.0635, -0.0068,
         0.1610, -0.0217,  0.6311, -0.0362], grad_fn=<ViewBackward>)

In [59]:
yb

tensor([1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1,
        0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0,
        0, 0, 0, 1])

In [60]:
preds.size()

torch.Size([52])

In [61]:
yb.size()

torch.Size([52])

In [62]:
accuracy_binomial(preds.to(device),yb.to(device))

tensor(0.5962)

In [63]:
from sklearn.metrics import roc_auc_score

In [64]:
roc_auc_score(yb,preds)

0.48660714285714285

In [65]:
pretrained_lm_weights

array([[-1.227409e-01,  2.788629e-01, -3.884962e-01,  3.981197e-01, ...,  3.707490e-01, -1.040416e-01,  1.957962e-02,
         1.854789e-01],
       [ 1.485352e-05, -2.342431e-05,  1.969303e-05, -2.154383e-05, ...,  2.051086e-05,  2.134880e-05,  2.177563e-05,
        -1.239415e-05],
       [-1.829635e-02, -1.382560e-01,  1.438132e-02, -1.285086e-02, ...,  3.653777e-03,  4.884117e-03,  5.742843e-02,
        -7.598962e-03],
       [-6.413675e-02,  1.201958e+00, -1.590474e-02,  5.468471e-02, ...,  2.948808e-02,  1.085662e-02, -8.994952e-01,
         3.026809e-01],
       ...,
       [-1.829635e-02, -1.382560e-01,  1.438132e-02, -1.285086e-02, ...,  3.653777e-03,  4.884117e-03,  5.742843e-02,
        -7.598962e-03],
       [-1.829635e-02, -1.382560e-01,  1.438132e-02, -1.285086e-02, ...,  3.653777e-03,  4.884117e-03,  5.742843e-02,
        -7.598962e-03],
       [-4.195510e-02, -2.414285e-01,  1.600470e-02, -1.684147e-02, ..., -1.989262e-02, -5.589788e-03,  9.034025e-02,
        -2.928584e

## Learner

In [66]:
# Weight Decay Schedule
tempstart=1e-7
for i in range (6):
    print (i, tempstart)
    tempstart=tempstart*6

0 1e-07
1 6e-07
2 3.6e-06
3 2.16e-05
4 0.0001296
5 0.0007776


In [67]:
tempstart=5e-3
for i in range (6):
    print (i, tempstart)
    tempstart=tempstart*(0.7**i)

0 0.005
1 0.005
2 0.0034999999999999996
3 0.0017149999999999995
4 0.0005882449999999997
5 0.0001412376244999999


In [68]:
torch.randint(1,5,(1,))

tensor([3])

In [69]:
lrs=[1e-2,5e-3,1e-4,5e-4]

In [70]:
torch.randint(0,4,(1,))

tensor([0])

In [71]:
lrs[torch.randint(0,4,(1,))]

0.0001

In [72]:
class Learner():
    def __init__(self,model,optimizer,metric_fn,device,bptt=12,print_every=5,clip_val=None,\
                 cycle_mult=0,lr_decay=0.7,wd_mult=6):
        self.model,self.optimizer,self.metric_fn,self.device,self.print_every,self.bptt,self.losses,self.clip_val=\
            model,optimizer,metric_fn,device,print_every,bptt,[],clip_val
        self.n_epochs=1
        self.cycle_mult,self.lr_decay=cycle_mult,lr_decay
        self.wd_mult=wd_mult
        for param_group in self.optimizer.param_groups:
            self.start_lr=param_group['lr']
            self.start_wd=param_group['weight_decay']
        self.wd=self.start_wd
        self.lr=self.start_lr
        self.n_epoch=0
        self.lrs=[1e-2,5e-3,1e-4,5e-4]
        self.preds,self.preds_valid,self.trainY,self.actual=[],[],[],[]
        
    def fit (self,Xb,Yb,Xlen,mode_train=True):
        if mode_train:
            self.model.train()
        else:
            self.model.eval()
            
        preds,loss=self.model(Xb,Yb,Xlen)
        
        with torch.no_grad():
            acc=self.metric_fn(preds,Yb.view(-1),self.device)
            acc=acc.item()
            
            if mode_train:
                self.trainY.append(Yb.view(-1))
                self.preds.append(preds.data)
            else:
                self.actual.append(Yb.view(-1))
                self.preds_valid.append(preds.data)
            
            del preds
        
        if mode_train:
            if 1==0:
                lr =self.lrs[torch.randint(0,4,(1,))]
                for param_group in self.optimizer.param_groups:
                    param_group['lr']=lr
            
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
        myloss=loss.item()
        del loss
        
        if self.clip_val is not None:
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.clip_val)
        
        return myloss, acc
    
    def lr_find (self,start_lr,end_lr,iterator,n_batch):
        losses,lrs=[],[]
        ratio=end_lr/start_lr
        num_steps=n_batch
        lr=start_lr
        for i in range(num_steps):            
            lr=lr*(end_lr/start_lr)**(1/num_steps)
            lrs.append(lr)
        self.lrs=lrs
        self.run_epoch(iterator,mode_train=True,lrs=lrs)
    
    def run_epoch(self,iterator,mode_train,lrs=None):
        epoch_loss,epoch_acc,i,k=0,0,0,0
        self.model.init_hidden()
        for Xb,Yb,Xlen in iterator:
            Xb=Xb.to(self.device)
            Yb=Yb.to(self.device)
            Xlen=Xlen.to(self.device)
            
            if lrs is not None:
                lr=lrs[k]
                for param_group in self.optimizer.param_groups:
                    param_group['lr']=lr 
            

            loss,acc=self.fit(Xb,Yb,Xlen,mode_train)
            
            if lrs is not None:
                self.losses.append(loss)
            
            
            epoch_loss+=loss
            epoch_acc+=acc
            
            k=k+1
            if k%self.print_every == 0:
                if k:
                    print (f'Batch:{k} {epoch_loss/(k)}  {epoch_acc/(k)}')  
                    torch.cuda.empty_cache()
        epoch_loss=epoch_loss/len(iterator)
        epoch_acc=epoch_acc/len(iterator)
            
        return epoch_loss,epoch_acc
    
    def plot_lrs(self, n_roll=1):
        import seaborn as sns
        ax=sns.lineplot(x=self.lrs,y=pd.Series(self.losses).rolling(n_roll).mean())
        ax.set_xscale('log')
        ax.set_ylabel('Loss')
        ax.set_xlabel('Learning Rate')

     
    def run_epochs(self,dltrain,dlvalid,n_epochs=1):
        
        if self.cycle_mult > 0:
            reset_cycle=self.cycle_mult
        
        for epoch in range(n_epochs):                

            
            loss,acc=self.run_epoch(dltrain,True)
            lossv,accv=self.run_epoch(dlvalid,mode_train=False)
            print (f'Epoch:{epoch} Learning rate {self.lr} Weight Decay {self.wd} Train Loss:{loss} Train Accuracy:{acc} Valid Loss:{lossv} Valid Accuracy:{accv}')
        
            if self.cycle_mult:
                if self.n_epoch==reset_cycle:
                    self.lr=self.start_lr
                    #self.wd=self.start_wd
                    reset_cycle=self.n_epoch+reset_cycle
                else:
                    self.lr*=(self.lr_decay**self.n_epoch)  
                    if self.n_epoch>1:
                        self.wd*=self.wd_mult
            self.n_epoch+=1
                
                
            for param_group in self.optimizer.param_groups:
                param_group['lr']=self.lr
                #param_group['weight_decay']=self.wd

In [73]:
device="cuda:1"

In [74]:
n_layers

2

In [75]:
model_sentiment=sentiment_classifier (n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e,dropout,\
                 dropout_o,pretrain_mtx=pretrained_lm_weights,n_out=1,padding_idx=1)
model_sentiment=model_sentiment.to(device)

initializing glove with (60002, 400)


In [76]:
optimizer=torch.optim.Adam(model_sentiment.parameters(),lr=5e-3,betas=(0.9,0.999), weight_decay=wd)
#optimizer=torch.optim.SGD(model_sentiment.parameters(),lr=1e-2,momentum=0.9, weight_decay=wd)
metric_fn=accuracy_binomial

In [77]:
len(dltrain),len(dlvalid)

(10770, 1150)

In [78]:
df_valid['label'].value_counts()

0    29986
1    29814
Name: label, dtype: int64

In [83]:
learner=Learner(model_sentiment,optimizer,accuracy_binomial,device,bptt,500,0.25,cycle_mult=6)

In [80]:
model_sentiment.freeze_embedding()

In [81]:
model_sentiment.encoder.weight.requires_grad, learner.model.encoder.weight.requires_grad

(False, False)

In [84]:
model_sentiment.dropout_e,model_sentiment.dropout,model_sentiment.dropout_o, learner.model.dropout_e,learner.model.dropout,learner.model.dropout_o

(0.2, 0.5, 0.5, 0.2, 0.5, 0.5)

In [85]:
learner.run_epochs(dltrain,dlvalid,5)

Batch:500 0.38001909692585467  0.8233077243566513
Batch:1000 0.3065233438424766  0.8645000350475311
Batch:1500 0.2745874199991425  0.8813333697716395
Batch:2000 0.2544250233210623  0.8915961907505989
Batch:2500 0.242202724955976  0.8977231141805648
Batch:3000 0.23303538528767725  0.9022051657040914
Batch:3500 0.2256694547432874  0.9058626750367028
Batch:4000 0.2201988925067708  0.9084856147021055
Batch:4500 0.21544423888002834  0.910837644788954
Batch:5000 0.21143236012868583  0.9127192687153817
Batch:5500 0.2079755813282999  0.9144860519495878
Batch:6000 0.20500842518235246  0.915910294353962
Batch:6500 0.20275546941676964  0.916985245172794
Batch:7000 0.20065681831059712  0.9180412468739918
Batch:7500 0.19890509835481643  0.9187949099063873
Batch:8000 0.19752617287402974  0.9193750381469726
Batch:8500 0.19608372873045943  0.9200814860848819
Batch:9000 0.19497819552115267  0.920606875816981
Batch:9500 0.19393129518000704  0.9211680543422699
Batch:10000 0.1931441226599738  0.9215173458

In [86]:
torch.save(model_sentiment.state_dict(),f'{DATAPATH}/inter/combo_model_dict_freeze')
torch.save(optimizer.state_dict(),f'{DATAPATH}/inter/combo_optimizer_dict_freeze')
torch.save (model_sentiment,f'{DATAPATH}/inter/combo_model_freeze')
torch.save (optimizer,f'{DATAPATH}/inter/combo_optimizer_freeze')
torch.save (learner,f'{DATAPATH}/inter/combo_learner_freeze')

/home/kirana/anaconda3/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type sentiment_classifier. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [87]:
from itertools import chain

In [102]:
preds=list(chain.from_iterable(learner.preds))[-df_train.shape[0]:]
preds_valid=list(chain.from_iterable(learner.preds_valid))[-df_valid.shape[0]:]
trainY=list(chain.from_iterable(learner.trainY))[-df_train.shape[0]:]
actual=list(chain.from_iterable(learner.actual))[-df_valid.shape[0]:]
preds=[x.item() for x in preds]
preds_valid=[x.item() for x in preds_valid]
trainY=[x.item() for x in trainY]
actual=[x.item() for x in actual]

In [103]:
from sklearn.metrics import roc_auc_score

In [104]:
roc_auc_score(trainY,preds),roc_auc_score(actual,preds_valid)

(0.9938809332844389, 0.9953131020186603)

In [105]:
from sklearn.metrics import precision_score, recall_score,f1_score

In [106]:
from scipy.special import expit

In [107]:
f1_score(trainY,np.round(expit(preds))), f1_score(actual,np.round(expit(preds_valid)))

(0.9626113608135035, 0.968318846866085)

In [108]:
precision_score(trainY,np.round(expit(preds))), precision_score(actual,np.round(expit(preds_valid)))

(0.9633775106866269, 0.9632887443157301)

In [109]:
recall_score(trainY,np.round(expit(preds))), recall_score(actual,np.round(expit(preds_valid)))

(0.9618464285714285, 0.9734017575635607)

In [97]:
model_sentiment.unfreeze_embedding()

In [98]:
model_sentiment.encoder.weight.requires_grad, learner.model.encoder.weight.requires_grad

(True, True)

In [99]:
learner.lr

0.0001412376244999999

In [100]:
learner=Learner(model_sentiment,optimizer,accuracy_binomial,device,bptt,500,0.25,cycle_mult=0)

In [101]:
learner.run_epochs(dltrain,dlvalid,2)

Batch:500 0.11832200812548399  0.9535000346899033
Batch:1000 0.1188365598898381  0.9524423427581787
Batch:1500 0.11943354298795263  0.9528590095440547
Batch:2000 0.11832190256332979  0.9537404196560383
Batch:2500 0.11737273623533546  0.9541461885213852
Batch:3000 0.11693801787060996  0.954243624428908
Batch:3500 0.11742168455570937  0.9541044302497591
Batch:4000 0.11713982405187562  0.9541058038473129
Batch:4500 0.11606956864582996  0.9544102909300063
Batch:5000 0.11545702863987535  0.9547961883902549
Batch:5500 0.11525633646496995  0.9549860484816811
Batch:6000 0.11464977006350334  0.955243624150753
Batch:6500 0.1138566996240272  0.9556272532022916
Batch:7000 0.11324705959404154  0.9558571770446641
Batch:7500 0.11296233197549979  0.9560205469528834
Batch:8000 0.11277249895385466  0.9561154186874629
Batch:8500 0.1126403840485522  0.9561357807131374
Batch:9000 0.11216753039457318  0.9564231109221776
Batch:9500 0.11188432920949631  0.9565688598469684
Batch:10000 0.11136372012114153  0.95

In [110]:
torch.save(model_sentiment.state_dict(),f'{DATAPATH}/inter/combo_model_dict_nofreeze')
torch.save(optimizer.state_dict(),f'{DATAPATH}/inter/combo_optimizer_dict_nofreeze')
torch.save (model_sentiment,f'{DATAPATH}/inter/combo_model_nofreeze')
torch.save (optimizer,f'{DATAPATH}/inter/combo_optimizer_nofreeze')
torch.save (learner,f'{DATAPATH}/inter/combo_learner_nofreeze')